In [138]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from requests import get, head
import json
import re
import os

In [139]:
common = ['URL', 'Not Relevant', '50s', '60s', '70s', '80s', '90s', '00s', '10s']
namesFinal = ['Experiments visualization', 'Portraits of science', 'Science and politics', 'Accelerators','Machines', 'Inside the lab', 'Outside the lab', 'Data processing', 'Media', 'CERN sceneries']
namesAlina = ['Experiments visualization', 'Portraits of science', 'Science and politics', 'Accelerators','Machines', 'Inside the lab', 'Outside the lab', 'Experiments', 'Data processing', 'Media', 'CERN sceneries', 'CERN under construction']
namesSelection = ['Experiments visualization', 'Portraits of science', 'Science and politics', 'Accelerators','Machines', 'Inside the lab', 'Outside the lab', 'EXPERIMENTS', 'Data processing', 'Media', 'CERN sceneries', 'CERN-Nursery school']

In [140]:
namesFinal

['Experiments visualization',
 'Portraits of science',
 'Science and politics',
 'Accelerators',
 'Machines',
 'Inside the lab',
 'Outside the lab',
 'Data processing',
 'Media',
 'CERN sceneries']

In [182]:
df = pd.read_excel('Images categorized.xlsx', sheet_name=0).replace(np.nan, 0)
df.columns = common + namesAlina
df['Inside the lab'] += df['Experiments']
df['CERN sceneries'] += df['CERN under construction']

df.drop(columns=['Experiments','CERN under construction'], inplace=True)

df2 = pd.read_excel('Images categorized.xlsx', sheet_name=1).replace(np.nan, 0)
df2.columns = common + namesFinal

df3 = pd.read_excel('Images to add.xlsx', sheet_name=0).replace(np.nan, 0)
df3.insert(1, 'Not Relevant', 0)
df3.columns = common + namesSelection
df3['Inside the lab'] += df3['EXPERIMENTS']
df3['Outside the lab'] += df3['CERN-Nursery school']
df3.drop(columns=['EXPERIMENTS', 'CERN-Nursery school'], inplace=True)
df3.columns

print("Same columns in dfs?", (df2.columns == df.columns).all() and (df.columns == df3.columns).all())
df = pd.concat([df, df2, df3]).reset_index(drop=True)

print('One decade per row?', (df.iloc[:, 2:9].sum(axis=1) == 1).all())
print(df.iloc[:, 9:19].columns)
one_category_filter = (df.iloc[:, 9:19].sum(axis=1) == 1)
print('One category per row?', one_category_filter.all())
no_category_filter = (df.iloc[:, 9:19].sum(axis=1) < 1)
print("There is %d entries with no category"%no_category_filter.sum())
# display(df[~one_category_filter].iloc[:, 9:19])
df = df[~no_category_filter]
df['decade'] = df.iloc[:, 2:9].idxmax(1)
df['category'] = list(map(lambda x: [y for y in x if isinstance(y, str)], df.iloc[:, 9:19].replace(1, pd.Series(df.columns, df.columns)).values.tolist()))

Same columns in dfs? True
One decade per row? False
Index(['Experiments visualization', 'Portraits of science',
       'Science and politics', 'Accelerators', 'Machines', 'Inside the lab',
       'Outside the lab', 'Data processing', 'Media', 'CERN sceneries'],
      dtype='object')
One category per row? False
There is 5 entries with no category


In [183]:
df = df[['decade', 'category', 'URL', 'Not Relevant']].replace(0, "")
def extract_id(s):
    return s.replace('http://cds.cern.ch/record/', '').replace('?ln=en', '').split()
df['Not Relevant'] = df['Not Relevant'].apply(extract_id)

In [184]:
df.head()

,decade,category,URL,Not Relevant
0,60s,[Inside the lab],http://cds.cern.ch/search?ln=en&cc=Photos&p=ex...,"[619810, 618882, 41165, 41067, 40995, 1759631,..."
1,60s,[Accelerators],http://cds.cern.ch/search?ln=en&cc=Photos&p=ac...,"[41444, 43706, 41444, 41046, 40344, 39973]"
2,60s,[Inside the lab],http://cds.cern.ch/search?ln=en&cc=Photos&p=li...,"[788515, 1221350, 1221356, 1221405, 1221352, 7..."
3,60s,[CERN sceneries],http://cds.cern.ch/search?ln=en&cc=Photos&p=si...,[40324]
4,60s,[CERN sceneries],http://cds.cern.ch/search?ln=en&cc=Photos&p=in...,"[41452, 41029]"


In [185]:
def format_url(url):
    url = re.sub('of=..&?','', url)
    url = re.sub('jrec=\d+&?', '', url)
    url = re.sub('rg=\d+&?', '', url)
    url.rstrip('&')
    url += '&of=recjson'
    url += '&rg=1000'
    return url
df['URL'] = df['URL'].apply(format_url)

In [187]:
with open('data_selection.json', 'r') as infile:
    data = json.load(infile)

In [188]:
# data = {}
missing = []
for url in df['URL'].values:
    if url not in data:
        r  = get(url)
        print('Fetching', url)
        if r.status_code == 200:
            data[url] = r.json()
        else:
            missing.append(url)

In [189]:
missing

[]

In [253]:
for url, res in data.items():
    if len(res) >  200:
        data[url] = []
        for jrec in range(1, 602, 200):
            url = url+'&jrec=%d'%jrec
            if url in data:
                continue
            r  = get(url+'&jrec=%d'%jrec)
            print('Fetching', url+'&jrec=%d'%jrec)
            if r.status_code == 200:
                data[url].extend(r.json())
            else:
                missing.append(url)

Fetching http://cds.cern.ch/search?ln=en&cc=Photos&p=accelerators+and+year%3A1980-%3E1989&action_search=Search&op1=a&m1=a&p1=&f1=&c=Photos&c=&sf=&so=d&rm=&sc=0&&of=recjson&rg=1000&jrec=1
Fetching http://cds.cern.ch/search?ln=en&cc=Photos&p=accelerators+and+year%3A1980-%3E1989&action_search=Search&op1=a&m1=a&p1=&f1=&c=Photos&c=&sf=&so=d&rm=&sc=0&&of=recjson&rg=1000&jrec=201
Fetching http://cds.cern.ch/search?ln=en&cc=Photos&p=accelerators+and+year%3A1980-%3E1989&action_search=Search&op1=a&m1=a&p1=&f1=&c=Photos&c=&sf=&so=d&rm=&sc=0&&of=recjson&rg=1000&jrec=401
Fetching http://cds.cern.ch/search?ln=en&cc=Photos&p=accelerators+and+year%3A1980-%3E1989&action_search=Search&op1=a&m1=a&p1=&f1=&c=Photos&c=&sf=&so=d&rm=&sc=0&&of=recjson&rg=1000&jrec=601
Fetching http://cds.cern.ch/search?ln=en&op1=a&p=experiments+and+detectors+year%3A1970-%3E1979&cc=Photos&m1=a&of=recjson&rg=1000&jrec=1
Fetching http://cds.cern.ch/search?ln=en&op1=a&p=experiments+and+detectors+year%3A1970-%3E1979&cc=Photos&m1=a&

In [191]:
df = pd.read_json('selection.json')

In [197]:
df[df['URL'] == url]['Not Relevant'].values

array([], dtype=object)

In [199]:
url_files = {}
not_found = []
for url, res in data.items():
    url_files[url] = []
    for r in res:
        found = False
        recid = r['recid']
        if str(recid) in df[df['URL'] == url]['Not Relevant'].values:
            continue
        if 'files' in r and len(r['files']) >= 1:
            for file in r['files']:
                if 'jpg' in file['url'] or 'jpeg' in file['url']:
                    url_files[url].append(file['url'])
                    found = True
                    break
            if found:
                continue
            for file in r['files']:
                if 'gif' in file['url']:
                    url_files[url].append(file['url'])
                    found = True
                    break
        if not found and 'electronic_location' in r:
            for location in r['electronic_location']:
                if 'uri' in location:
                    if '144' in location['uri']:
                        url_files[url].append(location['uri'])
                        found = True
                        break
            if not found:
                print(r['files'])
        if not found and 'primary_report_number' in r:
            photo_id = re.sub("\D", "", r['primary_report_number'])
            year = int(r['imprint']['date'])
            uri = "https://mediastream.cern.ch/MediaArchive/Photo/Public/%d/%s/%s/%s-A4-at-144-dpi.jpg"%(year, photo_id, photo_id, photo_id)
            req = head(uri)
            if req.status_code == 200:
                url_files[url].append(uri)
                found = True
            elif 'url' in r:
                photo_id_int = r['url'][0]['eformart'].split('/')[-1].split('_')[0]
                photo_id = re.sub("[^0-9_]", "", r['url'][0]['eformart'])
                year = int(r['creation_date'].split('-')[0])
                uri = "https://mediastream.cern.ch/MediaArchive/Photo/Public/%d/%s/%s/%s-A4-at-144-dpi.jpg"%(year, photo_id_int, photo_id, photo_id)
                req = head(uri)
                if req.status_code == 200:
                    url_files[url].append(uri)
                    found = True
        if not found:
            not_found.append((url, r))

In [201]:
len(not_found)

10

In [202]:
df_urls = pd.Series(url_files).reset_index().rename(columns={'index': 'URL', 0: 'url_list'})
df_urls['url_list'].apply(len).sum()

3231

In [203]:
df = df.set_index('URL').join(df_urls.set_index("URL")).reset_index()

In [204]:
df.to_json('selection.json')

In [205]:
for decade, categories, urls in df[['decade', 'category', 'url_list']].values:
    for category in categories:
        download_dir = './photos/' + '/'.join([decade, category]) + '/'
        if not os.path.exists(download_dir):
            os.makedirs(download_dir)
        print(download_dir)
        num_urls = len(urls)
        for idx, url in enumerate(urls):
            file_name = url.split('/')[-1]
            if os.path.exists(download_dir + file_name):
                continue
            with open(download_dir + file_name, 'wb') as outfile:
                res = get(url, stream=True)
                for chunk in res.iter_content(chunk_size=512):
                    if chunk:
                        outfile.write(chunk)
            print("Done with file %d/%d"%(idx+1, num_urls))

./photos/60s/Portraits of science/
./photos/60s/Outside the lab/
./photos/80s/Experiments visualization/
./photos/50s/Media/
./photos/70s/Machines/
./photos/70s/Machines/
./photos/50s/Media/
./photos/80s/Portraits of science/
./photos/80s/Inside the lab/
./photos/80s/Experiments visualization/
./photos/80s/Experiments visualization/
./photos/80s/Experiments visualization/
./photos/80s/Experiments visualization/
./photos/80s/Experiments visualization/
./photos/80s/Experiments visualization/
./photos/70s/Experiments visualization/
./photos/60s/Experiments visualization/
./photos/80s/Portraits of science/
./photos/80s/Outside the lab/
./photos/80s/CERN sceneries/
./photos/80s/Inside the lab/
./photos/70s/Machines/
./photos/50s/Media/
./photos/80s/Accelerators/
./photos/80s/Outside the lab/
./photos/70s/Machines/
./photos/80s/Portraits of science/
./photos/80s/Portraits of science/
./photos/60s/Science and politics/
./photos/60s/Portraits of science/
./photos/60s/Science and politics/
./ph

Done with file 129/465
Done with file 130/465
Done with file 131/465
Done with file 132/465
Done with file 136/465
Done with file 137/465
Done with file 139/465
Done with file 140/465
Done with file 141/465
Done with file 142/465
Done with file 143/465
Done with file 144/465
Done with file 147/465
Done with file 148/465
Done with file 149/465
Done with file 150/465
Done with file 151/465
Done with file 152/465
Done with file 153/465
Done with file 154/465
Done with file 155/465
Done with file 156/465
Done with file 157/465
Done with file 159/465
Done with file 160/465
Done with file 162/465
Done with file 163/465
Done with file 165/465
Done with file 166/465
Done with file 169/465
Done with file 170/465
Done with file 171/465
Done with file 173/465
Done with file 174/465
Done with file 176/465
Done with file 178/465
Done with file 179/465
Done with file 181/465
Done with file 182/465
Done with file 183/465
Done with file 185/465
Done with file 186/465
Done with file 189/465
Done with f

Done with file 1/41
Done with file 7/41
Done with file 15/41
Done with file 16/41
Done with file 20/41
Done with file 21/41
Done with file 23/41
Done with file 24/41
Done with file 25/41
./photos/60s/CERN sceneries/
Done with file 1/28
./photos/50s/Inside the lab/
Done with file 2/3
./photos/60s/Inside the lab/
Done with file 1/106
Done with file 6/106
Done with file 8/106
Done with file 10/106
Done with file 11/106
Done with file 12/106
Done with file 13/106
Done with file 14/106
Done with file 25/106
Done with file 27/106
Done with file 29/106
Done with file 32/106
Done with file 33/106
Done with file 38/106
Done with file 39/106
Done with file 40/106
Done with file 43/106
Done with file 45/106
Done with file 47/106
Done with file 53/106
Done with file 54/106
Done with file 59/106
Done with file 60/106
Done with file 66/106
Done with file 67/106
Done with file 81/106
Done with file 83/106
Done with file 84/106
Done with file 85/106
Done with file 87/106
Done with file 88/106
Done wit

In [206]:
with open('data_selection.json', 'w') as outfile:
    json.dump(data, outfile)

In [207]:
df['num_images'] = df['url_list'].apply(len)

In [208]:
df

,URL,Not Relevant,category,decade,url_list,num_images
0,http://cds.cern.ch/record/1221358?ln=en&of=rec...,[],[Portraits of science],60s,[http://cds.cern.ch/record/1221358/files/61032...,1
1,http://cds.cern.ch/record/1221405?ln=en&of=rec...,[],[Outside the lab],60s,[http://cds.cern.ch/record/1221405/files/64120...,1
2,http://cds.cern.ch/record/1271347?ln=en&of=rec...,[],[Experiments visualization],80s,[http://cds.cern.ch/record/1271347/files/81040...,1
3,http://cds.cern.ch/record/2055727?ln=en&of=rec...,[],[Media],50s,[http://cds.cern.ch/record/2055727/files/vol1-...,1
4,http://cds.cern.ch/record/39066?ln=en&of=recjs...,[],[Machines],70s,[https://mediastream.cern.ch/MediaArchive/Phot...,1
5,http://cds.cern.ch/record/39066?ln=en&of=recjs...,[],[Machines],70s,[https://mediastream.cern.ch/MediaArchive/Phot...,1
6,http://cds.cern.ch/record/39072?ln=en&of=recjs...,[],[Media],50s,[https://mediastream.cern.ch/MediaArchive/Phot...,1
7,http://cds.cern.ch/record/39348?ln=en&of=recjs...,[],"[Portraits of science, Inside the lab]",80s,[http://mediaarchive.cern.ch/MediaArchive/Phot...,1
8,http://cds.cern.ch/record/39454?ln=en&of=recjs...,[],[Experiments visualization],80s,[http://mediaarchive.cern.ch/MediaArchive/Phot...,1
9,http://cds.cern.ch/record/39455?ln=en&of=recjs...,[],[Experiments visualization],80s,[https://mediastream.cern.ch/MediaArchive/Phot...,1


In [212]:
df_flat = df.join(df['category'].apply(pd.Series).stack().reset_index(level=1, drop=True).to_frame("category_flat"))

In [213]:
df_flat[['decade', 'category_flat', 'num_images']].groupby('decade').sum()

,num_images
decade,
50s,342
60s,742
70s,1510
80s,674


In [215]:
df_flat[['decade', 'category_flat', 'num_images']].groupby('category_flat').sum()

,num_images
category_flat,
Accelerators,990
CERN sceneries,175
Data processing,56
Experiments visualization,20
Inside the lab,912
Machines,38
Media,15
Outside the lab,59
Portraits of science,816


In [217]:
df_flat[['decade', 'category_flat', 'num_images']].groupby(['decade', 'category_flat']).sum()

num_images
decade category_flat                        
50s    Accelerators                      101
       CERN sceneries                     57
       Experiments visualization           1
       Inside the lab                     21
       Machines                           22
       Media                              10
       Outside the lab                     1
       Portraits of science              123
       Science and politics                6
60s    Accelerators                      150
       CERN sceneries                     75
       Data processing                    19
       Experiments visualization           3
       Inside the lab                    284
       Machines                            2
       Outside the lab                    12
       Portraits of science              188
       Science and politics                9
70s    Accelerators                      465
       CERN sceneries                      4
       Data processing                     2
       Experiments visualization           2
       Inside the lab                    521
       Machines                           11
       Media                               3
       Outside the lab                     4
       Portraits of science              454
       Science and politics               44
80s    Accelerators                      274
       CERN sceneries                     39
       Data processing                    35
       Experiments visualization          14
       Inside the lab                     86
       Machines                            3
       Media                               2
       Outside the lab                    42
       Portraits of science               51
       Science and politics              128

In [226]:
df_long

,decade,category_flat,file_name
0,60s,Portraits of science,6103294.jpg
1,60s,Outside the lab,6412071.jpg
2,80s,Experiments visualization,8104099_01.jpg
3,50s,Media,vol1-issue1-covers.jpg?subformat=icon-1440
4,70s,Machines,7701602-A4-at-144-dpi.jpg
5,70s,Machines,7701602-A4-at-144-dpi.jpg
6,50s,Media,5310001-A4-at-144-dpi.jpg
7,80s,Portraits of science,8902126-A4-at-144-dpi.jpg
7,80s,Portraits of science,8902126-A4-at-144-dpi.jpg
7,80s,Inside the lab,8902126-A4-at-144-dpi.jpg


In [218]:
available_files = pd.read_csv('./available_files').values.reshape(-1)

In [221]:
df_flat['file_names'] = df_flat['url_list'].apply(lambda l: [url.split('/')[-1] for url in l])

In [223]:
df_flat

,URL,Not Relevant,category,decade,url_list,num_images,category_flat,file_names
0,http://cds.cern.ch/record/1221358?ln=en&of=rec...,[],[Portraits of science],60s,[http://cds.cern.ch/record/1221358/files/61032...,1,Portraits of science,[6103294.jpg]
1,http://cds.cern.ch/record/1221405?ln=en&of=rec...,[],[Outside the lab],60s,[http://cds.cern.ch/record/1221405/files/64120...,1,Outside the lab,[6412071.jpg]
2,http://cds.cern.ch/record/1271347?ln=en&of=rec...,[],[Experiments visualization],80s,[http://cds.cern.ch/record/1271347/files/81040...,1,Experiments visualization,[8104099_01.jpg]
3,http://cds.cern.ch/record/2055727?ln=en&of=rec...,[],[Media],50s,[http://cds.cern.ch/record/2055727/files/vol1-...,1,Media,[vol1-issue1-covers.jpg?subformat=icon-1440]
4,http://cds.cern.ch/record/39066?ln=en&of=recjs...,[],[Machines],70s,[https://mediastream.cern.ch/MediaArchive/Phot...,1,Machines,[7701602-A4-at-144-dpi.jpg]
5,http://cds.cern.ch/record/39066?ln=en&of=recjs...,[],[Machines],70s,[https://mediastream.cern.ch/MediaArchive/Phot...,1,Machines,[7701602-A4-at-144-dpi.jpg]
6,http://cds.cern.ch/record/39072?ln=en&of=recjs...,[],[Media],50s,[https://mediastream.cern.ch/MediaArchive/Phot...,1,Media,[5310001-A4-at-144-dpi.jpg]
7,http://cds.cern.ch/record/39348?ln=en&of=recjs...,[],"[Portraits of science, Inside the lab]",80s,[http://mediaarchive.cern.ch/MediaArchive/Phot...,1,Portraits of science,[8902126-A4-at-144-dpi.jpg]
7,http://cds.cern.ch/record/39348?ln=en&of=recjs...,[],"[Portraits of science, Inside the lab]",80s,[http://mediaarchive.cern.ch/MediaArchive/Phot...,1,Inside the lab,[8902126-A4-at-144-dpi.jpg]
8,http://cds.cern.ch/record/39454?ln=en&of=recjs...,[],[Experiments visualization],80s,[http://mediaarchive.cern.ch/MediaArchive/Phot...,1,Experiments visualization,[8211511-A4-at-144-dpi.jpg]


In [224]:
df_long = df_flat[['decade', 'category_flat']].join(df_flat['file_names'].apply(pd.Series).stack().reset_index(level=1, drop=True).to_frame("file_name"))

In [230]:
df_long = df_long.dropna()

In [231]:
df_long['file_path'] = df_long.apply(lambda c: './photos/' + '/'.join(c), axis=1)

/home/raph/.conda/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [232]:
df_long[df_long['file_path'].isin(available_files)].groupby('decade')['file_path'].apply(list).to_json('decades.json')

In [233]:
df_long[df_long['file_path'].isin(available_files)].groupby('category_flat')['file_path'].apply(list).to_json('categories.json')

In [234]:
df_long[df_long['file_path'].isin(available_files)].groupby(['decade', 'category_flat'])['file_path'].apply(list).to_json('decade-category.json')

In [245]:
df_long[df_long['file_path'].isin(available_files)].drop_duplicates(subset='file_path').groupby('category_flat').size().to_frame('count')

,count
category_flat,
Accelerators,990
CERN sceneries,173
Data processing,56
Experiments visualization,18
Inside the lab,909
Machines,37
Media,15
Outside the lab,59
Portraits of science,798


In [246]:
df_long[df_long['file_path'].isin(available_files)].drop_duplicates(subset='file_path').groupby('decade').size().to_frame('count')

,count
decade,
50s,341
60s,741
70s,1489
80s,666


In [247]:
df_long[df_long['file_path'].isin(available_files)].drop_duplicates(subset='file_path').groupby(['decade', 'category_flat']).size().to_frame('count')

count
decade category_flat                   
50s    Accelerators                 101
       CERN sceneries                57
       Experiments visualization      1
       Inside the lab                21
       Machines                      22
       Media                         10
       Outside the lab                1
       Portraits of science         123
       Science and politics           5
60s    Accelerators                 150
       CERN sceneries                74
       Data processing               19
       Experiments visualization      3
       Inside the lab               284
       Machines                       2
       Outside the lab               12
       Portraits of science         188
       Science and politics           9
70s    Accelerators                 465
       CERN sceneries                 4
       Data processing                2
       Experiments visualization      2
       Inside the lab               520
       Machines                      10
       Media                          3
       Outside the lab                4
       Portraits of science         437
       Science and politics          42
80s    Accelerators                 274
       CERN sceneries                38
       Data processing               35
       Experiments visualization     12
       Inside the lab                84
       Machines                       3
       Media                          2
       Outside the lab               42
       Portraits of science          50
       Science and politics         126